<a href="https://colab.research.google.com/github/nklsrmy/CCADMACL_EXERCISES_COM222ML/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1

Use all feature selection methods to find the best features

## Dataset Information

## Features

Number of Instances: 20640

Number of Attributes: 8 numeric, predictive attributes and the target

Attribute Information:

MedInc - median income in block group

HouseAge - median house age in block group

AveRooms - average number of rooms per household

AveBedrms - average number of bedrooms per household

Population - block group population

AveOccup - average number of household members

Latitude - block group latitude

Longitude - block group longitude

## Target
The target variable is the median house value for California districts, expressed in hundreds of thousands of dollars ($100,000).

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [2]:
housing = fetch_california_housing(as_frame=True)
df = pd.concat([housing.data, housing.target], axis=1)

In [3]:
df_housing_data =  pd.DataFrame(housing.data, columns=housing.feature_names)
df_housing_target = pd.DataFrame(housing.target, columns=['MedHouseVal'])

In [4]:
df.corr()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
MedInc,1.000000,-0.119034,0.326895,-0.062040,0.004834,0.018766,-0.079809,-0.015176,0.688075
HouseAge,-0.119034,1.000000,-0.153277,-0.077747,-0.296244,0.013191,0.011173,-0.108197,0.105623
AveRooms,0.326895,-0.153277,1.000000,0.847621,-0.072213,-0.004852,0.106389,-0.027540,0.151948
AveBedrms,-0.062040,-0.077747,0.847621,1.000000,-0.066197,-0.006181,0.069721,0.013344,-0.046701
Population,0.004834,-0.296244,-0.072213,-0.066197,1.000000,0.069863,-0.108785,0.099773,-0.024650
AveOccup,0.018766,0.013191,-0.004852,-0.006181,0.069863,1.000000,0.002366,0.002476,-0.023737
Latitude,-0.079809,0.011173,0.106389,0.069721,-0.108785,0.002366,1.000000,-0.924664,-0.144160
Longitude,-0.015176,-0.108197,-0.027540,0.013344,0.099773,0.002476,-0.924664,1.000000,-0.045967
MedHouseVal,0.688075,0.105623,0.151948,-0.046701,-0.024650,-0.023737,-0.144160,-0.045967,1.000000


In [5]:
df.corr()['MedHouseVal'].abs().sort_values(ascending=False)

,MedHouseVal
MedHouseVal,1.000000
MedInc,0.688075
AveRooms,0.151948
Latitude,0.144160
HouseAge,0.105623
AveBedrms,0.046701
Longitude,0.045967
Population,0.024650
AveOccup,0.023737


1. Use any filter method to select the best features

In [6]:
# put your answer here
from sklearn.feature_selection import mutual_info_regression

In [7]:
threshold = 5
high_score_features = []
feature_scores = mutual_info_regression(df_housing_data, df_housing_target.values.ravel())

In [8]:
for score, r_name in sorted(zip(feature_scores, df_housing_data.columns), reverse=True)[:threshold]:
    high_score_features.append(r_name)

df_housing_fr = df_housing_data[high_score_features]
df_housing_fr.columns

Index(['Longitude', 'MedInc', 'Latitude', 'AveRooms', 'AveOccup'], dtype='object')

2. Use any wrapper method to select the best features

In [9]:
# put your answer here
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

rf_model = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
selector = RFE(rf_model, n_features_to_select=4, step=1)

selector = selector.fit(df_housing_data, df_housing_target.values.ravel())
selector_ind = selector.get_support()
df_housing_rfe = df_housing_data.iloc[:, selector_ind]
df_housing_rfe.columns

Index(['MedInc', 'AveRooms', 'AveOccup', 'Latitude'], dtype='object')

3. Use any embedded methood to select the best features

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_housing_data, df_housing_target, test_size=0.2, random_state=42)

In [11]:
# put your answer here
from sklearn.feature_selection import SelectFromModel

rf_model = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
rf_model.fit(X_train, y_train.values.ravel())

sel_sfm = SelectFromModel(rf_model, prefit=True)
sel_sfm_index = sel_sfm.get_support()
df_housing_sfm = df_housing_data.iloc[:, sel_sfm_index]
df_housing_sfm.columns

Index(['MedInc', 'AveOccup'], dtype='object')

# Model Comparison

In [12]:
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
model = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [14]:
X_train_fr, X_test_fr, y_train_fr, y_test_fr = train_test_split(df_housing_fr, df_housing_target, test_size=0.2, random_state=42)

In [15]:
model_fr = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_fr.fit(X_train_fr, y_train_fr)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [16]:
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(df_housing_rfe, df_housing_target, test_size=0.2, random_state=42)

In [17]:
model_rfe = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_rfe.fit(X_train_rfe, y_train_rfe)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [18]:
X_train_sfm, X_test_sfm, y_train_sfm, y_test_sfm = train_test_split(df_housing_sfm, df_housing_target, test_size=0.2, random_state=42)

In [19]:
model_sfm = RandomForestRegressor(n_estimators=500, random_state=0, max_depth = 3)
model_sfm.fit(X_train_sfm, y_train_sfm)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=500, random_state=0)

In [20]:
default_preds = model.predict(X_test)
fr_preds = model_fr.predict(X_test_fr)
rfe_preds = model_rfe.predict(X_test_rfe)
sfm_preds = model_sfm.predict(X_test_sfm)

In [21]:
default_rmse = mean_squared_error(y_test, default_preds, squared=False)
fr_rmse = mean_squared_error(y_test, fr_preds, squared=False)
rfe_rmse = mean_squared_error(y_test, rfe_preds, squared=False)
sfm_rmse = mean_squared_error(y_test, sfm_preds, squared=False)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

In [22]:
print(f'Default RMSE: {default_rmse}')
print(f'Filter RMSE: {fr_rmse}')
print(f'Wrapper RMSE: {rfe_rmse}')
print(f'Embedded RMSE: {sfm_rmse}')

Default RMSE: 0.7750447841432082
Filter RMSE: 0.7752968572722918
Wrapper RMSE: 0.7753135462465947
Embedded RMSE: 0.7823934576498581
